In [1]:
import numpy as np 
import pandas as pd
import datetime

Reading Data

In [26]:
start_date = '2015-05-03'
end_date = '2015-05-30'

events = pd.read_csv('../data/events.csv')
events = events.drop_duplicates(subset=['timestamp', 'visitorid','itemid'], keep = 'first')
events = events.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in events.timestamp))
events = events.sort_values('date').reset_index(drop=True)
fd = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date()
events = events[(events.date >= fd(start_date)) & (events.date <= fd(end_date))]
events = events[['visitorid','itemid','event', 'date']]

#category_df - remove null parent ids
category_df = pd.read_csv('../data/category_tree.csv')


In [27]:
events.columns

Index(['visitorid', 'itemid', 'event', 'date'], dtype='object')

In [22]:
# events.duplicated(subset=['timestamp', 'visitorid','itemid'], keep = 'first')==True
print(events.shape)

print(events.shape)

(2756101, 5)
(2755630, 5)


In [23]:
# events[events.duplicated(subset=['timestamp', 'visitorid','itemid'], keep='first')==False]
events[events['visitorid']==257597]

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
152610,1433800821247,257597,view,302696,NaN
